In [2]:
import json
import re
import inflect
import pandas as pd
import spacy
from _complex import ComplexFunc
from py2neo import Graph
from py2neo.matching import *
import warnings
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore")
#     activated = spacy.prefer_gpu()
#     warnings.warn('default')
# activated

C:\Users\haris\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\haris\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\haris\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [3]:
class QuestionAnswer:
    """docstring for QuestionAnswer."""

    def __init__(self):
        super(QuestionAnswer, self).__init__()
        self.complex = ComplexFunc()
        self.nlp = spacy.load('en_core_web_lg')
        self.p = inflect.engine()
        self.graph = Graph('bolt://localhost:7687', auth = ("neo4j", "tester"))
        self.nodes_matcher = NodeMatcher(self.graph)
        self.relationship_matcher = RelationshipMatcher(self.graph)
    
    def answer(self, question):
        p=None
        try:
            p = self.complex.question_pairs(question.lower())[0]
        except: 
            print()
        if p == [] or p is None:
            quest = self.nlp(q)
            p = ['']*6
            for tok in quest:
                if tok.dep_ == 'compound' :
                        p[0] = tok.text
                elif tok.dep_ == 'nsubj' :
                    p[0] += ' ' + tok.text
                elif tok.dep_ == 'ROOT':
                    p[1] = tok.text
                elif tok.dep_ == 'attr':
                    p[-1] = tok.text.lower()


        if p[0] == 'unknown' or p[0] == '' or p[0] is None:
            quest = self.nlp(q)
            for tok in quest:
                if tok.ent_type_ == 'PERSON' :
                    if p[0] == 'unknown':
                        p[0] = tok.text
                    else: p[0] += ' ' + tok.text
        
        
        for i in range(len(p)):
            p[i] = p[i].replace('-', " ")

        if 'birth' in question.split():
            p[1] = 'be' 

        if str(p[1].lower()) == "born":
            qRel = 'be' 
        else:
            rel = self.nlp(p[1])
            for tok in rel:
                qRel = tok.lemma_
    

        objectQ = p[3]
        subList = []
        timeQ = str(p[4]).lower()
        placeQ = str(p[5]).lower()
        res = ''
        cypher = ''

        print(p)
        # Looking for a person or description of a person
        if 'who' in p:
            if objectQ != '':
                cypher = f'match (n)-[r:{qRel} | {p[1].lower()}]-(m) where n.name contains "{objectQ}" or n.name contains "{p[0].lower()}" return m.name'
            else:
                cypher = f'match (n)-[r:{qRel} | {p[1].lower()}]-(m) where n.name contains "{p[0].lower()}" return m.name'

            query = self.graph.run(cypher)
            if query != None:
                for node in query:
                    doc = self.nlp(str(node))
                    for i in doc:
                        if i.ent_type_ == "PERSON":
                            res += node[0] + ","
                        if i.ent_type_ == "NORP":
                            res += node[0] +','
                if res != '':
                    return res
                return query.forward()
            return "Answer Not Found"


        # Return a Location
        elif 'where' in p:
            cypher = f'match (n)-[r:{qRel} | {p[1].lower()}]-(m) where n.name contains "{p[0].lower()}" return m.name as Result'
            query = self.graph.run(cypher)
            if query != None:
                for node in query:
                    doc = self.nlp(str(node))
                    for i in doc:
                        if i.ent_type_ == "GPE":
                            res += node[0] + ","
                        # if i.ent_type_ == "NORP":
                        #     res += node[0] +','
                if res != '':
                    return res
                return query.forward()
            return "Answer Not Found"

        elif 'what' in p:
            splittedQ = question.split()
            if "place" in splittedQ:
                cypher = f'match (n)-[r:{qRel} | {p[1].lower()}]-(m) where n.name contains "{p[0].lower()}" return m.name as Result'
                query = self.graph.run(cypher)
                if query != 0:
                    for node in query:
                        doc = self.nlp(str(node))
                        for i in doc:
                            if i.ent_type_ == "GPE":
                                res += node[0] + ","
                            elif i.ent_type_ == "ORG":
                                res += node[0] + ","
                           
                    if res != '':
                        return res
                    return query
            else:
                cypher = f'match (n)-[r:{qRel} | {p[1].lower()}]-(m) where n.name contains "{p[0].lower()}" and "Thing" in m.node_labels return m.name as Result'

            return "Answer Not Found"


        else:
            if "name" in p:
                cypher = f'match (n)-[]-(m) where n.name contains "{p[4].lower()}" and ("Person" in m.node_labels or "Subject" in m.node_labels) return m.name'
                query = self.graph.run(cypher)
                if query != None:
                    names = []
                    for name in query:
                        doc = self.nlp(str(name))
                        for i in doc:
                            if i.ent_type_ == "PERSON":
                                if name.values() not in names:
                                    names.append(name.values())
                    return names

            else:
                cypher = f'match (n)-[r:{qRel.lower()} | {p[1].lower()}]-(m) where n.name contains "{p[0].lower()}"  return m.name'

                query = self.graph.run(cypher)
            
        print (cypher)
        return query

In [4]:
q = "What is Picasso birthplace?"
qp = QuestionAnswer()
ans = qp.answer(q)
ans

['picasso birthplace', 'is', '', 'what', '', '']


'Answer Not Found'

In [ ]:
for i in ans:
    print(i)

In [38]:
'18th-century'.split()
# doc= nlp("spanish painter")
# for tok in doc:
#     print(tok.ent_type_)

['18th-century']

In [16]:
nlp = spacy.load('en_core_web_lg')
doc = nlp(str("When was Van Gogh born?"))
for i in doc:
        print(i.text + " - \t "+ i.dep_)

When - 	 advmod
was - 	 auxpass
Van - 	 compound
Gogh - 	 nsubj
born - 	 ROOT
? - 	 punct


In [15]:
cp = ComplexFunc()
cp.question_pairs("When was vincent van gogh Born?")

UnboundLocalError: local variable 'subject' referenced before assignment